In [1]:
import os

# Set loglevel to suppress tensorflow GPU messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import re
from itertools import count

import numpy as np
import tensorflow as tf
from scipy.io import loadmat, savemat

from config import get_config_kACE
from PIL import Image
from pcolors import pcolors
# import Print_Colors as pcolors
# import Print_Colors

In [14]:
#############################################
def fetch(name, patch_size=100, **kwargs):
    """
        Input:
            name - dataset name, should be in DATASETS
            kwargs - config {key: value} pairs.
                     Key should be in DATASET_DEFAULT_CONFIG
        Output:
            training_data - tf.data.Dataset with (x, y, prior)
                            shapes like (inf, patch_size, patch_size, ?)
            evaluation_data - tf.data.Dataset with (x, y, change_map)
                              shapes (1, h, w, ?)
            channels - tuple (c_x, c_y), number of channels for domains x and y
    """
    print('fetch')
    # also get the change map
    x_im, y_im, target_cm = DATASETS[name](prepare_data[name])

    print(f"{pcolors.yellow}Loading dataset.. x,y,changemap", x_im.shape, y_im.shape, target_cm.shape, f"{pcolors.end}")

    # if cpu crop the 10% of the image (central cropping)
    if not tf.config.list_physical_devices("GPU"):
        dataset = [
            tf.image.central_crop(tensor, 0.1) for tensor in [x_im, y_im, target_cm]
        ]
    else:
        dataset = [x_im, y_im, target_cm]

    dataset = [tf.expand_dims(tensor, 0) for tensor in dataset]
    x, y = dataset[0], dataset[1]

    # print(f"{pcolors.red} dataset shape", dataset)

    print("dataset:", tf.reduce_min(x), tf.reduce_max(y), )
    evaluation_data = tf.data.Dataset.from_tensor_slices(tuple(dataset))

    c_x, c_y = x_im.shape[-1], y_im.shape[-1]


    return x, y, evaluation_data, (c_x, c_y)

#############################################
def _clip(image):
    """
        Normalize image from R_+ to [-1, 1].

        For each channel, clip any value larger than mu + 3sigma,
        where mu and sigma are the channel mean and standard deviation.
        Scale to [-1, 1] by (2*pixel value)/(max(channel)) - 1

        Input:
            image - (h, w, c) image array in R_+
        Output:
            image - (h, w, c) image array normalized within [-1, 1]
    """
    temp = np.reshape(image, (-1, image.shape[-1]))

    limits = tf.reduce_mean(temp, 0) + 3.0 * tf.math.reduce_std(temp, 0)
    for i, limit in enumerate(limits):
        channel = temp[:, i]
        channel = tf.clip_by_value(channel, 0, limit)
        ma, mi = tf.reduce_max(channel), tf.reduce_min(channel)
        channel = 2.0 * ((channel) / (ma)) - 1
        temp[:, i] = channel

    return tf.reshape(tf.convert_to_tensor(temp, dtype=tf.float32), image.shape)

#############################################
def _texas(clip=True):
    """ Load Texas dataset from .mat """
    mat = loadmat("data/Texas/Cross-sensor-Bastrop-data.mat")
    print(f"{pcolors.blue}",mat.keys())
    #It is composed by a Landsat 5 TM as the pre-event image and a Landsat 5 TM, a EO-1 ALI and a Landsat 8 as post-event images. We provide also the ground truth we prepared to evaluate the method.
    # 't1_L5', 't2_L5', 't2_ALI', 't2_L8', 'ROI_2', 'ROI_1'
    t1 = np.array(mat["t1_L5"], dtype=np.single) #pre-event
    t2 = np.array(mat["t2_ALI"], dtype=np.single) 
    roi = mat["ROI_1"]
    print("texas: (before clip)", t1.shape, t2.shape, roi.shape)
    if clip:
        print("clipping")
        t1, t2 = _clip(t1), _clip(t2)
        print(f"{pcolors.cyan}texas (after clip):", t1.shape, t2.shape, f"{pcolors.end}")
    change_mask = tf.convert_to_tensor(mat["ROI_1"], dtype=tf.bool)
    assert t1.shape[:2] == t2.shape[:2] == change_mask.shape[:2]
    if change_mask.ndim == 2:
        change_mask = change_mask[..., np.newaxis]
    return t1, t2, change_mask

#############################################
DATASETS = {
    "Texas": _texas,
    # "California": _california,
    # "France": _france,
    # "Italy": _italy,
    # "UK": _uk,
    # "Denmark": _denmark,
}
prepare_data = {
    "Texas": True,
    "California": True,
    "France": True,
    "Italy": False,
    "UK": True,
    "Denmark": False,
}

In [15]:
# How is it used
DATASET="Texas"
CONFIG = get_config_kACE(DATASET)
print(CONFIG)
x_im, y_im, EVALUATE, (C_X, C_Y) = fetch(DATASET, **CONFIG)
# mat = tf.squeeze(x_im).numpy()
print(x_im.shape, y_im.shape)
# img = Image.fromarray(mat[:,:,1] , 'L')
# img.show()


{'debug': False, 'clipnorm': 1, 'learning_rate': 0.0001, 'l2_lambda': 1e-06, 'logdir': 'logs/Texas/20210107-115629', 'save_images': True, 'channel_x': [1, 2, 3], 'channel_y': [3, 4, 7], 'filter_': <function decorated_median_filter.<locals>.median_filter2d at 0x7fc6c9f38ea0>, 'final_filter': <function decorated_gaussian_filter.<locals>.gauss_filter at 0x7fc6f0081950>, 'patience': 10, 'minimum improvement': 0.001, 'list_epochs': [2, 2, 1], 'batches': 2, 'batch_size': 2, 'patch_size': 10, 'affinity_batch_size': 10, 'affinity_patch_size': 20, 'affinity_stride': 5, 'epochs': 5, 'cycle_lambda': 1, 'cross_lambda': 1, 'recon_lambda': 1, 'kernels_lambda': 1}
fetch
 dict_keys(['__header__', '__version__', '__globals__', 't1_L5', 't2_L5', 't2_ALI', 't2_L8', 'ROI_2', 'ROI_1'])
texas: (before clip) (1534, 808, 7) (1534, 808, 10) (1534, 808)
clipping
texas (after clip): (1534, 808, 7) (1534, 808, 10) 
Loading dataset.. x,y,changemap (1534, 808, 7) (1534, 808, 10) (1534, 808, 1) 
texas (before return

In [32]:
# #float32
# print(x_im.shape, y_im.shape, C_X, C_Y)
# print(tf.math.reduce_min(x_im))
# print(tf.math.reduce_max(x_im))
# x_im.shape[:-1]
# Pu = tf.expand_dims(tf.ones(x_im.shape[:-1], dtype=tf.float32), -1) #everyone but the last axix
# #creates an input of 1s like (1, 154, 82, 1)
# print(Pu.shape)
# print(x_im.shape, y_im.shape, C_X, C_Y)
# print("EVALUATE", EVALUATE) #master, slave, ground truth pixels (true false)
# print(tf.math.reduce_min(x_im))
# print(tf.math.reduce_max(x_im))


(1, 154, 82, 7) (1, 154, 82, 10) 7 10
tf.Tensor(-0.7616062, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
(1, 154, 82, 1)
(1, 154, 82, 7) (1, 154, 82, 10) 7 10
EVALUATE <TensorSliceDataset shapes: ((154, 82, 7), (154, 82, 10), (154, 82, 1)), types: (tf.float32, tf.float32, tf.bool)>
tf.Tensor(-0.7616062, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
